In [22]:
import os
import pandas as pd

import re


In [21]:
is_resultfile = lambda filename: re.search(r'^spark_.*\.csv$', filename) is not None
files = list(filter(is_resultfile, os.listdir()))
files

['spark_GBTClassificationModel.csv',
 'spark_LogisticRegressionModel.csv',
 'spark_DecisionTreeClassificationModel.csv',
 'spark_RandomForestClassificationModel.csv']

In [23]:
pd.read_csv(files[0])

,accuracy,precision,recall,auc,f1_macro,f1_micro,f1_weighted,binary,beta,tp,fn,fp,tn,y_test,y_predictions,model_name,hyperparameters
0,0.899611,0.819457,0.685983,0.822194,0.842099,0.899611,0.896261,0.746803,0.709082,876,401,193,4447,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",GBTClassificationModel,"[('cacheNodeIds', False), ('checkpointInterval..."
